In [130]:
# imputer+ouo+no靖妤+(bert)
# bert+pca(1??)
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = pd.read_csv('ML_FP_train_v6.csv')
print(data.columns)
print(data)
y = data['Danceability']
# data['Composer'] = data['Composer'].fillna(0)
# chart0c=data[['Composer','Danceability']]
# chart0c=chart0c.groupby('Composer',as_index=False).mean().sort_values('Danceability',ascending=False)
# chart0c['index'] = [float(i) for i in range(11)]
# label_mapping = chart0c.set_index('Composer')['index'].to_dict()
data = data[['Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Views', 'Likes', 'Stream', 'Album_type', 'Licensed',
       'official_video', 'Composer', 'Artist']]


# Encode labels in df1 to integers using the label_mapping dictionary
#data['Composer'] = data['Composer'].map(label_mapping)

sub_inf = pd.read_csv('ML_FP_test_v6.csv')
print(sub_inf)
tttest = sub_inf[['Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Views', 'Likes', 'Stream', 'Album_type', 'Licensed',
       'official_video', 'Composer', 'Artist']]
# tttest['Composer'] = tttest['Composer'].map(label_mapping)
X = data.copy()
sub_iid = pd.read_csv('ML_FP_test_v6.csv')
sub_id  = sub_iid['id']
merged = pd.concat([X, tttest], ignore_index=True)
print(merged.shape)

imputer = IterativeImputer(random_state=0) # 補植的方法

# Fit and transform the data
imputed_data = imputer.fit_transform(merged)

# Convert the imputed data back to a dataframe
imputed_ = pd.DataFrame(imputed_data, columns=data.columns)


split_index = len(X)  # Index to split the DataFrame

X_prebert = imputed_.iloc[:split_index]
tttest_prebert = imputed_.iloc[split_index:]

bb = pd.read_csv('bert_all.csv')
imputed_df = pd.concat([imputed_, bb], axis=1)
X = imputed_df.iloc[:split_index]
tttest = imputed_df.iloc[split_index:]

Index(['Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Views', 'Likes', 'Stream', 'Composer', 'Artist',
       'Licensed', 'official_video', 'Album_type'],
      dtype='object')
       Danceability    Energy  Key   Loudness  Speechiness  Acousticness  \
0               0.0  0.000273  3.0 -25.176000       0.0443  7.241508e-01   
1               0.0  0.184220  7.0 -12.390073          NaN  6.722214e-01   
2               0.0  0.054337  6.0 -15.596000          NaN  6.676276e-01   
3               1.0  0.209585  NaN  -6.251000       0.0277  3.796416e-03   
4               4.0       NaN  5.0  -6.219846       0.0260  3.048625e-06   
...             ...       ...  ...        ...          ...           ...   
17165           3.0  0.794023  5.0  -6.344000       0.0328  8.991539e-02   
17166           2.0  0.820026  4.0  -1.786000          NaN  2.195200e-05   
17167           1.0  0.57178

(23485, 18)


In [132]:
print(X_prebert.shape)
print(X.shape)
print(tttest_prebert.shape)
print(tttest.shape)
print(y.shape)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train_nb, x_test_nb, y_train, y_test = train_test_split(X_prebert, y, test_size=0.2, random_state=42)

(17170, 18)
(17170, 38)
(6315, 18)
(6315, 38)
(17170,)


In [133]:
def rround(y_):
    y_ = [round(num) for num in y_]
    #y_ = [int(x) + 1 if x > int(x) + 0.7 else int(x) if x < int(x) + 0.3 else int(x) + 0.5 for x in y_]
    y_ = [9 if x > 9 else x for x in y_]
    y_ = [0 if x < 0 else x for x in y_]
    return y_

In [134]:
test_t = pd.read_csv('test_partial_answer.csv')
def test_acc(df1, df2):
    merged_df = pd.merge(df1, df2, on='id', how='inner')

    # Calculate accuracy (absolute difference)
    merged_df['Accuracy'] = abs(merged_df['Danceability_x'] - merged_df['Danceability_y'])

    # Calculate average accuracy
    average_accuracy = merged_df['Accuracy'].mean()

    #print(merged_df)
    print("Average Accuracy:", average_accuracy)

In [135]:
n_best = pd.read_csv('ols_sc_rb_submission.csv')
test_acc(test_t, n_best)

Average Accuracy: 2.15689381933439


# RF

In [111]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf = rf_model.fit(x_train, y_train)
y_pred = rf.predict(x_test)
#print(rf_model.feature_names_in_)

In [112]:
print(len(y_train))
y_pred = rf.predict(x_test)
#print(rf_model.feature_names_in_)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mse = mean_squared_error(y_test, rround(y_pred))
print("Mean Squared Error2:", mse)

#no bert Mean Squared Error: 4.186648776936517
# bert
# Mean Squared Error: 4.235486051252184
# Mean Squared Error2: 4.2778101339545715
# pca bert
# Mean Squared Error: 4.16308683750728
# Mean Squared Error2: 4.216074548631334

13736
Mean Squared Error: 4.210959784507863
Mean Squared Error2: 4.299650553290623


In [113]:
print(len(X))
print(len(tttest))
rf = rf_model.fit(X, y)

y_submit = rf.predict(tttest)
print(y_submit)
#v1
#Mean Squared Error: 4.38743715783343
#[4.2  5.45 4.97 ... 3.87 4.43 5.08]
y_ = y_submit.tolist()

17170
6315
[5.45 5.81 3.25 ... 3.79 4.22 4.81]


In [114]:
y_ = rround(y_)

df = pd.DataFrame({'id': sub_id, 'Danceability': y_})
test_acc(df, test_t)
print(df)
df.to_csv('output0511_0417.csv', index=False)

Average Accuracy: 2.1822503961965136
         id  Danceability
0     17170             5
1     17171             6
2     17172             3
3     17173             5
4     17174             6
...     ...           ...
6310  23480             2
6311  23481             7
6312  23482             4
6313  23483             4
6314  23484             5

[6315 rows x 2 columns]


In [115]:
# no bert

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

rf = rf_model.fit(x_train_nb, y_train)
y_pred = rf.predict(x_test_nb)
#print(rf_model.feature_names_in_)

#print(rf_model.feature_names_in_)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mse = mean_squared_error(y_test, rround(y_pred))
print("Mean Squared Error2:", mse)

rf = rf_model.fit(X_prebert, y)
y_submit = rf.predict(tttest_prebert)
print(y_submit)
#v1
#Mean Squared Error: 4.38743715783343
#[4.2  5.45 4.97 ... 3.87 4.43 5.08]
y_ = y_submit.tolist()
df = pd.DataFrame({'id': sub_id, 'Danceability': y_})
test_acc(df, test_t)

Mean Squared Error: 4.252183779848573
Mean Squared Error2: 4.300815375655213
[5.52 6.38 3.86 ... 3.59 4.15 3.99]
Average Accuracy: 2.180316957210777


# xgboost

In [136]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
xg5 = xgb.XGBRegressor(n_estimators= 1600, max_depth= 9, learning_rate= 0.01,colsample_bytree= 0.5)#(XGBRegressor(colsample_bytree= 0.6, learning_rate=0.03, max_depth= None, n_estimators=1200)
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xg5=xg5.fit(x_train, y_train)
y_pred=xg5.predict(x_test)
mse2 = mean_squared_error(y_test, y_pred)
y_pred = rround(y_pred)
mse = mean_squared_error(y_test, y_pred)
print(mse)
print(mse2)
# no bert
# 3.9178800232964472
# 3.8325537957190416
# bert
# 3.768782760629004
# 3.6961818585097235
# pca bert
# 3.8692486895748397
# 3.795434217386355
# ouo+bert+imputer
# 3.7009318578916717
# 3.6424231889231544
# ouo+bert+imputer+tune
# 3.7009318578916717
# 3.6334081875442323

3.7501456027955737
3.661324162190281


In [137]:
xg5 = xgb.XGBRegressor(n_estimators= 1600, max_depth= 9, learning_rate= 0.01,colsample_bytree= 0.5)#((colsample_bytree= 0.6, learning_rate=0.03, max_depth= None, n_estimators=1200)
xg5 = xg5.fit(X, y)

In [138]:
print(len(tttest))
y_submit = xg5.predict(tttest)

6315


In [139]:

y_ = y_submit.tolist()

df = pd.DataFrame({'id': sub_id, 'Danceability': y_})
test_acc(df, test_t)
print(df)
df.to_csv('output0511_0418.csv', index=False)

y_ = rround(y_)

df = pd.DataFrame({'id': sub_id, 'Danceability': y_})
print(df)
df.to_csv('output0511_0419.csv', index=False)
test_acc(df, test_t)

Average Accuracy: 2.1322850617336586
         id  Danceability
0     17170      5.559577
1     17171      5.038537
2     17172      4.107034
3     17173      4.035712
4     17174      5.152710
...     ...           ...
6310  23480      1.745009
6311  23481      7.659138
6312  23482      3.500906
6313  23483      5.426163
6314  23484      5.238286

[6315 rows x 2 columns]
         id  Danceability
0     17170             6
1     17171             5
2     17172             4
3     17173             4
4     17174             5
...     ...           ...
6310  23480             2
6311  23481             8
6312  23482             4
6313  23483             5
6314  23484             5

[6315 rows x 2 columns]
Average Accuracy: 2.115689381933439


In [127]:
#no_bert




xg5 = xgb.XGBRegressor(n_estimators= 1600, max_depth= 9, learning_rate= 0.01,colsample_bytree= 0.5)#((colsample_bytree= 0.6, learning_rate=0.03, max_depth= None, n_estimators=1200)
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xg5=xg5.fit(x_train_nb, y_train)
y_pred=xg5.predict(x_test_nb)
mse2 = mean_squared_error(y_test, y_pred)
y_pred = rround(y_pred)
mse = mean_squared_error(y_test, y_pred)
print(mse)
print(mse2)
#no bert3.9178800232964472
#3.8325537957190416

3.8296447291788
3.753448690368451


In [128]:
xg5 = xgb.XGBRegressor(n_estimators= 1600, max_depth= 9, learning_rate= 0.01,colsample_bytree= 0.5)#(colsample_bytree= 0.6, learning_rate=0.03, max_depth= None, n_estimators=1200)
xg5 = xg5.fit(X_prebert, y)
y_submit = xg5.predict(tttest_prebert)

y_ = y_submit.tolist()
df = pd.DataFrame({'id': sub_id, 'Danceability': y_})

In [129]:
df.to_csv('output0511_0420.csv', index=False)
test_acc(df, test_t)
print(df)
y_ = rround(y_)

df = pd.DataFrame({'id': sub_id, 'Danceability': y_})
print(df)
df.to_csv('output0511_0421.csv', index=False)
test_acc(df, test_t)
# Average Accuracy: 2.121677498981097
# Average Accuracy: 2.114104595879556

Average Accuracy: 2.1282848111234265
         id  Danceability
0     17170      5.212711
1     17171      5.324709
2     17172      4.113466
3     17173      3.561276
4     17174      5.447711
...     ...           ...
6310  23480      1.637880
6311  23481      7.554855
6312  23482      3.323893
6313  23483      5.330450
6314  23484      5.163568

[6315 rows x 2 columns]
         id  Danceability
0     17170             5
1     17171             5
2     17172             4
3     17173             4
4     17174             5
...     ...           ...
6310  23480             2
6311  23481             8
6312  23482             3
6313  23483             5
6314  23484             5

[6315 rows x 2 columns]
Average Accuracy: 2.1378763866877972


In [89]:
# 大tune特tune
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_depth = [int(x) for x in np.linspace(1, 11, num=11)]
max_depth.append(None)
learning_rate=[round(float(x),2) for x in np.linspace(start=0.01, stop=0.2, num=10)]
colsample_bytree =[round(float(x),2) for x in np.linspace(start=0.1, stop=1, num=10)]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'learning_rate': learning_rate,
               'colsample_bytree': colsample_bytree}
random_grid 

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, None],
 'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.12, 0.14, 0.16, 0.18, 0.2],
 'colsample_bytree': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}

In [90]:
xg4 = xgb.XGBRegressor(random_state=42)

#Random search of parameters, using 3 fold cross validation, search across 100 different combinations, and use all available cores
xg_random = RandomizedSearchCV(estimator = xg4, param_distributions=random_grid,
                              n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

xg_random.fit(x_train_nb,y_train)
xg_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

# pca bert cell_1->this->cell_2

In [131]:
import numpy as np
from sklearn.decomposition import PCA

# Assuming arr is your NumPy array
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Create a PCA 
nofpca = 20
pca = PCA(n_components=nofpca)
xx = bb.values
print(xx.shape)
# Fit the PCA model to the data
pca.fit(xx)

# Transform the data using the PCA model
transformed_data = pca.transform(xx)

# Access the principal components
principal_components = pca.components_

# Access the explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_

# Print the transformed data
print("Transformed Data:")
print(transformed_data)
print(transformed_data.shape)

transformed_data = pd.DataFrame(transformed_data, columns=[str(i) for i in range(nofpca)])
imputed_df = pd.concat([imputed_, transformed_data], axis=1)
X = imputed_df.iloc[:split_index]
tttest = imputed_df.iloc[split_index:]
# Print the principal components
print("Principal Components:")
print(principal_components)

# Print the explained variance ratio
print("Explained Variance Ratio:")
print(explained_variance_ratio)


(23485, 128)
Transformed Data:
[[ 0.74965274  4.91019383  0.01421809 ... -1.00853796 -0.54940566
  -0.55335962]
 [ 2.17563501  1.80990828 -1.06249165 ... -0.67291899 -1.46211201
  -0.19950925]
 [ 2.17563501  1.80990828 -1.06249165 ... -0.67291899 -1.46211201
  -0.19950925]
 ...
 [-1.43822298  4.7593385  -2.19635838 ... -0.60114172 -0.40384402
  -1.60416858]
 [-3.0855156  -2.08006222 -3.27427677 ... -0.41621601  0.0837697
  -0.13250717]
 [ 8.64635101 -1.26154942 -0.29134027 ... -0.08465516 -0.03545006
   0.10121821]]
(23485, 20)
Principal Components:
[[-7.41173800e-02  1.15611832e-01 -4.32474879e-02 ... -5.31673806e-02
   7.29493311e-02 -1.47207944e-01]
 [-3.06940178e-02 -9.64543776e-02 -1.76173898e-01 ...  5.53533181e-03
   2.99629477e-02  2.41818511e-01]
 [ 3.34576357e-02  1.90813964e-02 -6.19551199e-02 ...  8.31613063e-02
   1.01425975e-01  5.17694625e-02]
 ...
 [ 3.85869471e-02 -1.41067425e-02  1.00002278e-01 ...  9.15998102e-02
   3.53387999e-02  6.27252477e-02]
 [ 1.27127917e-04  

In [78]:
X.to_csv('train_ouo_bert_pca3.csv', index=False)
tttest.to_csv('test_ouo_bert_pca3.csv',index=False)
X_prebert.to_csv('train_ouo_imputer.csv', index=False)
tttest_prebert.to_csv('test_ouo_imputer.csv',index=False)

(23485, 128)
